# 数据简介

数据是由超星平台提供

共39个表格，包括了各数据包字段说明、学生信息、课程信息、在线时长、课堂学生操作流水表、成绩表等数据

每个表基本只涵盖一类信息，所以最后我们需要将预处理后的信息汇总

因为数据量大，我们针对每个表单独进行删除或补充异常值、补充缺省值、删除无用数据，并将每个预处理好的数据重新保存为.CSV文件格式的数据表，避免仿佛处理浪费算力，将39表格的内容筛选汇总成9个表格

## 观察数据的函数

In [1]:
# 判断数据是否一致
def function(a, b):
    if a == b:
        # 一致输出1
        return 1
    else:
        # 不一致输出0
        return 0


# 判断insert_time create_time last_modify_time 时间列是否一致
# da-临时储存数据的变量
def ifon(df):
    da = df[['insert_time', 'create_time', 'last_modify_time']]
    da['bool1'] = da.apply(lambda x : function(x['insert_time'], x['create_time']), axis = 1)
    da['bool2'] = da.apply(lambda x : function(x['create_time'], x['create_time']), axis = 1)
    print(len(da[da['bool1'] == 0]), len(da[da['bool2'] == 0]))
    
# 展示各列具体信息
def data_col(data):
    for k in data.columns:
        print(k +'   '+ str(len(data[k].unique())))

## 数据预处理的函数

In [2]:
import os
import pandas as pd
from datetime import *

# 判断 personid 是否是学生id
def ifstud(data, data_stu, name1, name2, per = [], tea = []):
    '''
    data: 需要判断的数据集
    data_stu: 判断依据的数据集(学生名单)
    name1: 判断的列名-data-personid
    name2: 对比的列名-data_stu-personid
    return: per-存放学生的id-列表、tea-存放老师的id-列表
    '''
    for i in data[name1].unique():
        if i in data_stu[name2].unique():
            per.append(i)
        else:
            tea.append(i)
    return per, tea

# 删除非学生操作
def dropdata(data, inds):
    '''
    data: 需要处理的数据集
    inds: 需要删除的数据行的索引列表
    return: data-处理完毕的数据集
    '''
    data = data.drop(inds)
    # 重新设置索引 从0开始
    data.reset_index(drop = True, inplace = True)
    return data

def dropdata_(data, col, perid):
    '''
    data: 需要处理的数据集
    col: 匹配删除条件的列名(如:personid、role等)
    perid: 删除的条件(如:老师的personid列表)
    return: data-处理完毕的数据集
    '''
    if type(perid) == type(1):
        data = data.drop(data[data[col] == perid].index)
    elif len(perid) == 1:
        data = data.drop(data[data[col] == perid].index)
    else:
        for i in perid:
            data = data.drop(data[data[col] == i].index)
    # 重新设置索引 从0开始
    data.reset_index(drop = True, inplace = True)
    return data

# 计算时长
def get_hour(date1, date2):
    '''
    date1: 开始时间-str-datetime
    date2: 结束时间-str-datetime
    return: hour-时长(s)/0
    '''
    try:
        date1 = datetime.strptime(date1, "%Y-%m-%d %H:%M")
        date2 = datetime.strptime(date2, "%Y-%m-%d %H:%M")
        date = date2 - date1
        hour = date.seconds/3600
        return hour
    except:
        return 0
    
# 字符串转时间
def cgdatetime(df, name):
    '''
    df: 数据集名
    name: 列名
    return: df[name]
    '''
    df[name] = pd.to_datetime(df[name], format = '%Y-%m-%d %H:%M')
    return df[name]
    
# 数据去空去重
def df_clear(df):
    '''
    df: 数据集名
    return: 去空去重后的数据集
    '''
    df = df.drop_duplicates(inplace = False)
    df = df.dropna(axis = 0, how = 'all', inplace = False)
    # 重新设置索引 从0开始
    df.reset_index(drop = True, inplace = True)
    return df

# 数据清洗
# 删除每个字符串左侧、右侧中空格（包括换行符）
def data_str_clear(data):
    '''
    data: 数据集名
    return: 处理好的数据集
    '''
    for i in data.columns:
        try:
            data[i] = data[i].str.strip()
        except:
            pass
    return data

    
# 遍历一次性获得某个文件夹下的所有指定的文件路径
def FilesPath(path):
    '''
    path: 目录文件夹地址
    return: filePaths-文件全路径-列表
    '''
    filePaths = [] # 存储目录下的所有文件名，含路径
    for root,dirs,files in os.walk(path):
        for file in files:
            filePaths.append(os.path.join(root, file))
    return filePaths


def concatcsv(path):
    '''
    path: 需要遍历的文件夹路径
    '''
    # 调用FilesPath函数获得文件路径
    log_file = FilesPath(path)
    data_log = pd.DataFrame()
    for file_path in (log_file):
        dlog = pd.read_csv(file_path, encoding = 'utf-8')
        data_log = pd.concat([data_log, dlog])
    return data_log


# 单个文件的读取数据
def getdf(path, filename):
    '''
    path: 文件路径-如: 'data/t_stat_activity_log.xls'
    filename: 文件名-文件类型(只支持execl和csv文件)
    return: df-数据集
    '''
    if filename[-1] == 'v':
        df = pd.read_csv(path + filename, encoding = 'utf-8')
    else:
        df = pd.read_excel(path + filename)
    print(df.shape)
    return df

# 选取列保留
def getdata(df, colname, newcolname):
    '''
    df: 源数据集
    colname: 保留的列名列表
    newcolname: 新列名
    return: data-新数据集
    '''
    try:
        data = df[colname]
        data.columns = newcolname
        print(data.shape)
        return data
    except:
        print('列名错误')
    

# 判断是否有删除记录
def IFisDeleted(data):
    '''
    data: 数据集
    return: data-删除记录后的数据集
    '''
    if len(data['isDeleted'].unique()) > 1:
        i1 = data[data['isDeleted'] == 1][['personid', 'courseid', 'clazzid']].personid
        i2 = data[data['isDeleted'] == 1][['personid', 'courseid', 'clazzid']].courseid
        i3 = data[data['isDeleted'] == 1][['personid', 'courseid', 'clazzid']].clazzid
        data = data.drop(data[(data.personid == i1) & (data.courseid == i2) & (data.clazzid == i3)].index)
        # 重新设置索引 从0开始
        data.reset_index(drop = True, inplace = True)
    return data

def querydate(df, start, end, colname, newcolname):
    '''
    df: 待处理的数据集
    colname: 保留的列名列表
    newcolname: 新列名
    return: data
    '''
    data = df.loc[(df.last_modify_time >= start) & (df.last_modify_time <= end), colname]
    data.columns = newcolname
    # 重新设置索引 从0开始
    data.reset_index(drop = True, inplace = True)
#     print(data.shape)
    return data

def droptead(data, col, teaid, dropteaid = []):
    '''
    data: 数据集
    col: 条件列名(如:personid、用户id)
    teaid: 老师的用户id列表
    return: data
    '''
    for i in data.groupby(col):
        if i[0] in teaid:
            dropteaid += data[data[col] == i[0]].index.to_list()
        else:
            continue
        
    if len(dropteaid) != 0:
        data = dropdata(data, dropteaid)
        # 重新设置索引 从0开始
        data.reset_index(drop = True, inplace = True)
    else:
        pass
    return data

In [3]:
# 路径
path = 'data/'

# 日期范围
dates = ['2022-02-01', '2022-08-30']

# dtype对应的活动类型
acts = {'AttendLog':'签到', 'VoteLog':'投票', 'SurveyLog':'问卷', 'WorkLog':'测验', 'ChooseSomeoneLog':'选人', 'PreampAnswerLog':'抢答', 'MarkLog':'评分', 'TaskLog':'分组', 'askLog':'分组', 'LiveLog':'课堂直播', 'CourseNoticeLog':'课程通知', 'FeedbackLog':'群聊', 'SyncClassroomLog':'同步课堂', 'DiscussLog':'讨论', 'TimerLog':'计时器活动', 'ResourceActivitLog':'资源型活动', 'OnlineCourseLog':'在线课堂', 'TTencentMeetingLog':'在线课堂', 'OtherLog':'其他活动'}
print(acts)

{'AttendLog': '签到', 'VoteLog': '投票', 'SurveyLog': '问卷', 'WorkLog': '测验', 'ChooseSomeoneLog': '选人', 'PreampAnswerLog': '抢答', 'MarkLog': '评分', 'TaskLog': '分组', 'askLog': '分组', 'LiveLog': '课堂直播', 'CourseNoticeLog': '课程通知', 'FeedbackLog': '群聊', 'SyncClassroomLog': '同步课堂', 'DiscussLog': '讨论', 'TimerLog': '计时器活动', 'ResourceActivitLog': '资源型活动', 'OnlineCourseLog': '在线课堂', 'TTencentMeetingLog': '在线课堂', 'OtherLog': '其他活动'}


In [4]:
# 存放老师的personid
df_pers = getdf(path, 't_stat_person.xls')
df_perb = getdf(path, 'b1_t_stat_person.xlsx')

# 老师的用户id
perids = df_pers[df_pers['role'] == 1].personid.to_list()
peridb = df_perb[(df_perb['role'] == 0) | (df_perb['role'] == 2)].personid.to_list()
print(len(peridb))

df_pers = df_pers[df_pers['role'] == 3]
df_perb = df_perb[df_perb['role'] == 3]
peridb += df_perb[df_perb.user_name.str.contains(r'[0-9a-zA-Z]') == True].personid.to_list()
print(len(peridb))
print(df_pers.shape, df_perb.shape)

(1513, 10)
(40581, 9)
6429
7106
(1485, 10) (34152, 9)


## 1. b1_t_stat_clazz --> 班级表

In [5]:
# 雨课堂的数据
df_cla = getdf(path, 'b1_t_stat_clazz.xlsx')
# 筛选日期2022.2.1-2022.8.30的数据行
data_cla = querydate(df_cla, dates[0], dates[1], ['courseid', 'clazzid'], ['课程id', '班级id'])
print(data_cla.shape)

# 数据清洗
data_cla = data_str_clear(data_cla)
print(data_cla.shape)
data_cla.head()

(24280, 13)
(12937, 2)
(12937, 2)


,课程id,班级id
0,1943082,5372193
1,1943082,5372194
2,1943083,5372195
3,1943081,5372196
4,1943084,5372197


## 2. 学生名单

In [6]:
# 学习通的数据
data_pers = getdata(df_pers, ['personid', *df_pers.columns[5:-2]], ['用户id', 'uid', '姓名', '学号'])
data_pers = df_clear(data_pers)
data_pers = data_pers[data_pers['姓名'].str.contains(r'[0-9a-zA-Z]') == False]
data_pers = data_pers[data_pers['姓名'].str.contains(r'[\u4e00-\u9fa5]') == True]
data_pers.reset_index(drop = True, inplace = True)
print(data_pers.shape)

# 雨课堂的数据
df_perb = dropdata_(df_perb, 'role', [0, 2])

# 筛选日期2022.2.1-2022.8.30的数据行
data_perb = querydate(df_perb, dates[0], dates[1], [*df_perb.columns[4:6], 'personid'], ['姓名', '学号', '用户id'])
data_perb = df_clear(data_perb)
data_perb = data_perb[data_perb['姓名'].str.contains(r'[0-9a-zA-Z]') == False]
data_perb = data_perb[data_perb['姓名'].str.contains(r'[\u4e00-\u9fa5]') == True]
data_perb.reset_index(drop = True, inplace = True)
print(data_perb.shape)
# display(data_pers.head(), data_perb.head())

(1485, 4)
(1485, 4)
(32761, 3)


In [7]:
df_tm = getdf(path, 'duration.xlsx')
data_tm = getdata(df_tm, ['用户id（uid）', '在线时长（秒）'], ['uid', '在线时长（秒）'])
data_tm.head()

(29414, 3)
(29414, 2)


,uid,在线时长（秒）
0,147025170,451753
1,55039653,32436
2,147022961,252447
3,103131505,314077
4,82832356,48804


In [8]:
for i in data_pers.uid.unique():
    if i not in data_tm.uid.unique():
        k = data_pers[data_pers.uid == i].index.values
        data_pers.loc[data_pers.uid == i, 'uid'] = data_tm.loc[k[0], 'uid']

In [9]:
data_npers = pd.merge(data_pers, data_tm, on = 'uid')
data_npers = data_npers[['用户id', '姓名', '学号', '在线时长（秒）']]
data_npers.reset_index(drop = True, inplace = True)
print(data_npers.shape)
data_npers.head()

(1485, 4)


,用户id,姓名,学号,在线时长（秒）
0,51465331,丁屹中,20161003949,7047
1,111944914,娜迪热·艾尔肯,20191004561,209251
2,111946468,何倩,20191004398,207885
3,147372556,熊锐,20201001447,334696
4,147374240,郑玉洁,20201001777,118632


In [10]:
tmc = pd.concat([data_tm, data_tm.loc[:32760-29414, :]], axis = 0)
tmc.reset_index(drop = True, inplace = True)
data_perb.loc[:, '在线时长（秒）'] = tmc['在线时长（秒）']
data_perb.head()

,姓名,学号,用户id,在线时长（秒）
0,丁磊,20201000062,28405756,451753
1,严佳旺,20201000327,28405787,32436
2,付少坤,20201004172,28405855,252447
3,任勇浩,20201002222,28405870,314077
4,伏锦曦,20201001817,28405892,48804


## 2. t_stat_course_person --> 人课关系表

In [11]:
# 学习通的数据
df_cps = getdf(path, 't_stat_course_person.xls')
df_cps = dropdata_(df_cps, 'role', 1)
df_cps = dropdata_(df_cps, 'clazzid', 0)

# 判断是否有删除记录并删除
df_cps = IFisDeleted(df_cps)
data_cps = getdata(df_cps, ['personid', 'courseid'], ['用户id', '课程id'])
data_cps.head()

(2284, 10)
(2246, 2)


,用户id,课程id
0,51465331,226410295
1,111944914,226410295
2,111946468,226410295
3,147372556,226410295
4,147374240,226410295


In [12]:
# 雨课堂的数据
# 获得所以文件路径
cpb_file = FilesPath('data/b1_t_stat_course_person/')
df_cpb = pd.DataFrame()
for file_path in (cpb_file):
    dlog = pd.read_csv(file_path, encoding = 'utf-8')
    df_cpb = pd.concat([df_cpb, dlog])

# 重新设置索引 从0开始
df_cpb.reset_index(drop = True, inplace = True)
df_cpb = dropdata_(df_cpb, 'role', 1)
df_cpb = dropdata_(df_cpb, 'clazzid', 0)

print(df_cpb.shape)
# 判断是否有删除记录并删除
df_cpb = IFisDeleted(df_cpb)
# 筛选日期2022.2.1-2022.8.30的数据行
data_cpb = querydate(df_cpb, dates[0], dates[1], ['personid', 'clazzid'], ['用户id', '班级id'])
print(data_cpb.shape)
data_cpb.head()

(551548, 10)
(328400, 2)


,用户id,班级id
0,28428481,8009920
1,28428482,8009920
2,28428483,8009920
3,28428484,8009920
4,28428485,8009920


In [13]:
data_cpb = pd.merge(data_cpb, data_cla, on = '班级id')
data_cpb.reset_index(drop = True, inplace = True)
print(data_cpb.shape)

(195420, 3)


In [14]:
# 数据合并
data_cp = pd.concat([data_cps, data_cpb[['用户id', '课程id']]], axis = 0)
data_cp = df_clear(data_cp)
data_cp.reset_index(drop = True, inplace = True)
print(data_cp.shape)
data_cp.head()

(79750, 2)


,用户id,课程id
0,51465331,226410295
1,111944914,226410295
2,111946468,226410295
3,147372556,226410295
4,147374240,226410295


#### 数据合并

In [554]:
data_stu = pd.concat([data_npers, data_perb], axis = 0)
data_stu = data_stu[data_stu['用户id'].isna() == False]
data_stu = data_stu[data_stu['姓名'].isna() == False]
data_stu = data_str_clear(data_stu)
data_stu.reset_index(drop = True, inplace = True)
print(data_stu.shape)
data_stu.head()

(34246, 4)


,用户id,姓名,学号,在线时长（秒）
0,51465331,丁屹中,20161003949,7047
1,111944914,娜迪热·艾尔肯,20191004561,209251
2,111946468,何倩,20191004398,207885
3,147372556,熊锐,20201001447,334696
4,147374240,郑玉洁,20201001777,118632


#### 保存数据

In [556]:
data_stu.to_csv('E:/比赛/data/1.学生名单.csv', index = False, encoding = 'utf_8_sig')

## 3. t_stat_activity_log.xls --> 课堂活动参与表

In [18]:
# 学习通的数据
df = pd.read_excel('data/t_stat_activity_log.xls')
# col_name-用于暂存需要列的名称
col_name = ['attend_time', *df.columns[4:5].values, *df.columns[6:8], 'attend_id', 'status']

# 提取出所需的列
data = df[col_name]


key = [0, 1, 2, 4, 5, 7, 8, 9, 10, 11, 12]
val = ['未签', '已签', '代签', '请假', '缺勤', '病假', '事假', '迟到', '早退', '签到已过期', '公假']
sets = dict(zip(key, val))
data.loc[data['dtype'] == 'AttendLog', 'dtype'] = data[data['dtype'] == 'AttendLog'].status.map(sets)

# 数据清洗
data = dropdata_(data, 'personid', perids)
data = data_str_clear(data)
# 删除状态
data.pop('status')

# 改列名
colnames = ['日期', '活动类型', '用户id', '课程id', '活动id']
data.columns = colnames

print(data['活动类型'].unique())
print(data.shape)
data.head()

['VoteLog' 'WorkLog' 'PreampAnswerLog' 'ChooseSomeoneLog' 'TaskLog' '已签'
 '签到已过期' '代签' '公假' '病假']
(6129, 5)


,日期,活动类型,用户id,课程id,活动id
0,2022-04-25 14:47,VoteLog,111949230,224841013,6000022019599
1,2022-04-25 14:47,VoteLog,111947461,224841013,6000022019599
2,2022-04-25 14:48,VoteLog,111944124,224841013,6000022019599
3,2022-04-25 14:48,VoteLog,111944820,224841013,6000022019599
4,2022-04-25 14:48,VoteLog,111947022,224841013,6000022019599


#### 雨课堂数据

In [19]:
# 雨课堂的数据
df_b = pd.read_csv('data/b1_t_stat_activity_log.csv', chunksize = 10000)
data_b = pd.DataFrame()
# 每块数据 30w 行
for i in df_b:
    # 筛选日期2022.2.1-2022.8.30的数据行
    i1 = querydate(i, dates[0], dates[1], ['attend_time', 'dtype', 'personid', 'clazzid'], ['日期', '活动类型', '用户id', '班级id'])
    # 数据清洗
    i1 = data_str_clear(i1)
    data_b = pd.concat([data_b, i1])
print(data_b.shape)
data_b = droptead(data_b, '用户id', peridb)
data_b['活动类型'] = '已签'
data_b['日期'] = cgdatetime(data_b, '日期').apply(lambda x:x.strftime('%Y-%m-%d %H:%M')) 
data_b.drop_duplicates(subset = ['日期', '活动类型', '用户id', '班级id'],inplace = True)
data_b['活动id'] = '其他'
print(data_b.shape)
data_b.head()

(419493, 4)
(382672, 5)


,日期,活动类型,用户id,班级id,活动id
0,2021-09-02 07:56,已签,28405756,6136781,其他
1,2021-09-02 08:55,已签,28405756,6136781,其他
2,2021-09-03 09:51,已签,28405756,6137477,其他
3,2021-09-09 10:06,已签,28405756,6137100,其他
4,2022-03-09 14:52,已签,28405855,8011388,其他


In [20]:
data_b = pd.merge(data_b, data_cla, on = '班级id')
print(data_b.shape)

(338503, 6)


#### 数据合并

In [21]:
# 数据合并
data_per = pd.concat([data, data_b[data.columns]], axis = 0)
data_per.reset_index(drop = True, inplace = True)
print(data_per.shape)
data_per.head()

(344632, 5)


,日期,活动类型,用户id,课程id,活动id
0,2022-04-25 14:47,VoteLog,111949230,224841013,6000022019599
1,2022-04-25 14:47,VoteLog,111947461,224841013,6000022019599
2,2022-04-25 14:48,VoteLog,111944124,224841013,6000022019599
3,2022-04-25 14:48,VoteLog,111944820,224841013,6000022019599
4,2022-04-25 14:48,VoteLog,111947022,224841013,6000022019599


#### 数据保存

In [557]:
data_per.to_csv('E:/比赛/data/3.学生课堂活动参与表.csv', index = False, encoding = 'utf_8_sig')

## 4. t_stat_bbs_log.xls --> 讨论流水
### 207 行 14 列

In [23]:
# 学习通数据
df_bbs = getdf(path, 't_stat_bbs_log.xls')
df_bbs = dropdata_(df_bbs, 'role', 1)
data_bbs = getdata(df_bbs, ['last_modify_time', 'personid', 'courseid', 'bbs_id'], ['日期', '用户id', '课程id', '活动id'])
data_bbs.insert(loc = 1, column = '活动类型', value = '讨论')
data_bbs.head()

(207, 14)
(205, 4)


,日期,活动类型,用户id,课程id,活动id
0,2022-06-15 15:09,讨论,68286748,224841013,201f62ceb08934c089e6a162c7759ac5
1,2022-06-15 15:10,讨论,68286748,224841013,201f62ceb08934c089e6a162c7759ac5
2,2022-06-15 15:15,讨论,68286748,224841013,201f62ceb08934c089e6a162c7759ac5
3,2022-06-15 15:20,讨论,68286748,224841013,201f62ceb08934c089e6a162c7759ac5
4,2022-06-15 15:24,讨论,68286748,224841013,201f62ceb08934c089e6a162c7759ac5


#### 雨课堂数据

In [24]:
# 雨课堂的数据
df_bbb = getdf(path, 'b1_t_stat_bbs_log.xlsx')
# 筛选日期2022.2.1-2022.8.30的数据行
data_bbb = querydate(df_bbb, dates[0], dates[1], ['last_modify_time', 'personid', 'clazzid', 'topic_id'], ['日期', '用户id', '班级id', '活动id'])
print(data_bbb.shape)

# 数据清洗
data_bbb = dropdata_(data_bbb, '用户id', peridb)
data_bbb = data_str_clear(data_bbb)
data_bbb.insert(loc = 1, column = '活动类型', value = '讨论')

print(data_bbb.shape)
data_bbb.head()

(1919, 14)
(1206, 4)
(1096, 5)


,日期,活动类型,用户id,班级id,活动id
0,2022-02-11 05:00:05,讨论,28433407,4200560,1486035
1,2022-02-11 05:00:05,讨论,28422471,5374085,3060165
2,2022-02-11 05:00:05,讨论,28422513,5374085,3060752
3,2022-02-11 05:00:05,讨论,28433351,4200560,1654964
4,2022-02-11 05:00:05,讨论,28433350,4200560,1654964


In [25]:
data_bbb = pd.merge(data_bbb, data_cla, on = '班级id')
data_bbb.drop_duplicates(inplace = True)
print(data_bbb.shape)

(1025, 6)


#### 数据合并

In [26]:
# 数据合并
data_bb = pd.concat([data_bbs, data_bbb[data_bbs.columns]], axis = 0)
data_bb.reset_index(drop = True, inplace = True)
print(data_bb.shape)
data_bb.head()

(1230, 5)


,日期,活动类型,用户id,课程id,活动id
0,2022-06-15 15:09,讨论,68286748,224841013,201f62ceb08934c089e6a162c7759ac5
1,2022-06-15 15:10,讨论,68286748,224841013,201f62ceb08934c089e6a162c7759ac5
2,2022-06-15 15:15,讨论,68286748,224841013,201f62ceb08934c089e6a162c7759ac5
3,2022-06-15 15:20,讨论,68286748,224841013,201f62ceb08934c089e6a162c7759ac5
4,2022-06-15 15:24,讨论,68286748,224841013,201f62ceb08934c089e6a162c7759ac5


#### 数据保存

In [558]:
data_bb.to_csv('E:/比赛/data/4.学生课堂讨论流水表.csv', index = False, encoding = 'utf_8_sig')

## 5. t_stat_exam_answer --> 学生考试提交记录表

In [28]:
df_ans = getdf(path, 't_stat_exam_answer.xls')
data_ans = getdata(df_ans, ['answer_time', 'personid', 'courseid', 'work_id', 'score', 'status'], ['日期', '用户id', '课程id', '活动id', '分数', '提交状态'])
data_ans = data_str_clear(data_ans)
data_ans = dropdata_(data_ans, '用户id', perids)
ls = data_ans[data_ans['提交状态'] == 1].index.to_list()
data_ans.insert(1, '活动类型', '测验')
data_ans.loc[data_ans['提交状态'] == 1, '活动类型'] = '未测'
data_ans.loc[data_ans['提交状态'] == 1, '日期'] =  df_ans.loc[ls,'last_modify_time']
data_ans.pop('提交状态')
data_ans.head()

(2179, 17)
(2179, 6)


C:\Users\SUGA\AppData\Local\Temp\ipykernel_19160\1081961814.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = data[i].str.strip()


,日期,活动类型,用户id,课程id,活动id,分数
0,2022-03-01 17:02,测验,147392291,222820410,147473875,100.0
1,2022-03-01 17:17,测验,111944017,222820410,147473875,100.0
2,2022-03-01 17:02,测验,147392786,222820410,147473875,90.0
3,2022-03-01 17:05,测验,111948433,222820410,147473875,90.0
4,2022-03-01 17:03,测验,147383116,222820410,147473875,50.0


In [29]:
df_exs = pd.read_excel('data/t_stat_exam_library.xls')
df_exres = pd.read_excel('data/t_stat_exam_relation.xls')
data_exs = df_exs[['courseid', 'paper_library_id', 'title']]
data_ers = df_exres[['courseid', 'paper_library_id']]
da_exs = pd.merge(data_ers, data_exs, on = ['courseid', 'paper_library_id'])
da_exs.columns = ['课程id', '活动id', '活动名']
da_exs.head()

,课程id,活动id,活动名
0,222820410,147473875,单元测试1（2022上）
1,222820410,147479140,单元测试2（2022上）
2,222820410,156228047,单元测试4（2022上）
3,222820410,156228261,单元测试5（2022上）
4,222820410,156228524,单元测试6（2022上）


In [30]:
d_exs = pd.merge(data_ans, da_exs, on = ['课程id', '活动id'])
d_exs = df_clear(d_exs)
d_exs.reset_index(drop = True, inplace = True)
d_exs.drop_duplicates()
print(d_exs.shape)
d_exs.head()

(2179, 7)


,日期,活动类型,用户id,课程id,活动id,分数,活动名
0,2022-03-01 17:02,测验,147392291,222820410,147473875,100.0,单元测试1（2022上）
1,2022-03-01 17:17,测验,111944017,222820410,147473875,100.0,单元测试1（2022上）
2,2022-03-01 17:02,测验,147392786,222820410,147473875,90.0,单元测试1（2022上）
3,2022-03-01 17:05,测验,111948433,222820410,147473875,90.0,单元测试1（2022上）
4,2022-03-01 17:03,测验,147383116,222820410,147473875,50.0,单元测试1（2022上）


#### 雨课堂数据

In [31]:
# 雨课堂的数据
df_anb = getdf(path, 'b1_t_stat_exam_answer.xlsx')
data_anb = querydate(df_anb, dates[0], dates[1], ['answer_time', 'personid', 'clazzid', 'exam_id', 'score'], ['日期', '用户id', '班级id', '活动id', '分数'])

# 数据清洗
data_anb = droptead(data_anb, '用户id', peridb)
data_anb = data_str_clear(data_anb)

data_anb.insert(1, '活动类型', '测验')
data_anb.head()

(83452, 17)


,日期,活动类型,用户id,班级id,活动id,分数
0,2022-05-06 10:06:00,测验,28405855,8009388,210080,80.0
1,2021-09-15 19:30:10,测验,28406016,6137475,113289,80.0
2,2021-09-27 19:20:06,测验,28406016,6137475,116994,100.0
3,2021-09-29 19:20:10,测验,28406016,6137475,117499,60.0
4,2021-11-01 18:50:18,测验,28406016,6137475,128205,60.0


In [32]:
df_erb = getdf(path, 'b1_t_stat_exam_relation.xlsx')
data_erb = querydate(df_erb, dates[0], dates[1], ['clazzid', 'exam_id', 'score'], ['班级id', '活动id', '总分'])
data_erb.head()

(1251, 15)


,班级id,活动id,总分
0,5375152,100785,100.0
1,7245084,134795,100.0
2,6138605,936960,9.0
3,6138604,931379,9.0
4,6137475,114898,100.0


In [33]:
data_ab = pd.merge(data_anb, data_cla, on = '班级id')
data_ab_ = pd.merge(data_ab, data_erb, on = ['班级id', '活动id'])
print(data_ab_.shape)
data_ab_.head()

(32860, 8)


,日期,活动类型,用户id,班级id,活动id,分数,课程id,总分
0,2022-05-06 10:06:00,测验,28405855,8009388,210080,80.0,1944089,100.0
1,2022-05-06 10:05:33,测验,28406687,8009388,210080,90.0,1944089,100.0
2,2022-05-06 10:06:00,测验,28412716,8009388,210080,100.0,1944089,100.0
3,2022-05-06 10:09:13,测验,28420546,8009388,210080,50.0,1944089,100.0
4,2022-05-06 10:05:41,测验,28428765,8009388,210080,90.0,1944089,100.0


In [34]:
for i in data_ab_[data_ab_['总分'] != 100]['总分'].unique():
    data_ab_.loc[data_ab_['总分'] == i, '分数'] =  ((data_ab_.loc[data_ab_['总分'] == i, '分数'])/i) * 100

In [35]:
data_nab = data_ab_[['日期', '活动类型', '用户id', '课程id', '活动id', '分数']]
print(data_nab.shape)
data_nab.head()

(32860, 6)


,日期,活动类型,用户id,课程id,活动id,分数
0,2022-05-06 10:06:00,测验,28405855,1944089,210080,80.0
1,2022-05-06 10:05:33,测验,28406687,1944089,210080,90.0
2,2022-05-06 10:06:00,测验,28412716,1944089,210080,100.0
3,2022-05-06 10:09:13,测验,28420546,1944089,210080,50.0
4,2022-05-06 10:05:41,测验,28428765,1944089,210080,90.0


#### 数据合并

In [36]:
# 数据合并
data_an = pd.concat([data_ans, data_nab], axis = 0)
data_an.reset_index(drop = True, inplace = True)
print(data_an.shape)
data_an.head()

(35039, 6)


,日期,活动类型,用户id,课程id,活动id,分数
0,2022-03-01 17:02,测验,147392291,222820410,147473875,100.0
1,2022-03-01 17:17,测验,111944017,222820410,147473875,100.0
2,2022-03-01 17:02,测验,147392786,222820410,147473875,90.0
3,2022-03-01 17:05,测验,111948433,222820410,147473875,90.0
4,2022-03-01 17:03,测验,147383116,222820410,147473875,50.0


## 6. t_stat_course --> 课程表

In [37]:
df_cs = getdf(path, 't_stat_course.xls')
data_cs = getdata(df_cs, ['courseid', 'name', 'chapter_count'], ['课程id', '课程名', '课程章节数'])
data_cs = data_str_clear(data_cs)
data_cs.head()

(20, 12)
(20, 3)


C:\Users\SUGA\AppData\Local\Temp\ipykernel_19160\1081961814.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = data[i].str.strip()


,课程id,课程名,课程章节数
0,222595437,岩溶水文地质学,5
1,222602451,液压传动,71
2,222652756,高等数学B2,5
3,222675513,水力学,98
4,222760546,智能传感与检测技术,36


#### 雨课堂数据

In [38]:
df_cb = getdf(path, 'b1_t_stat_course.xlsx')
data_cb = querydate(df_cb, dates[0], dates[1], ['courseid','name'], ['课程id', '课程名'])
data_cb.head()

(3888, 12)


,课程id,课程名
0,1763844,中西科幻影视文学欣赏
1,1763852,形体训练与健美
2,1763853,户外救援
3,1763854,户外运动组织与管理
4,1763857,运动生理学


#### 数据合并

In [39]:
data_c = pd.concat([data_cs[['课程id', '课程名']], data_cb], axis = 0)
data_c.reset_index(drop = True, inplace = True)
data_c.head()

,课程id,课程名
0,222595437,岩溶水文地质学
1,222602451,液压传动
2,222652756,高等数学B2
3,222675513,水力学
4,222760546,智能传感与检测技术


In [40]:
data_exam = pd.merge(data_an, data_c, on = '课程id')
data_exam.reset_index(drop = True, inplace = True)
print(data_exam.shape)
data_exam.head()

(32881, 7)


,日期,活动类型,用户id,课程id,活动id,分数,课程名
0,2022-03-01 17:02,测验,147392291,222820410,147473875,100.0,数据库原理
1,2022-03-01 17:17,测验,111944017,222820410,147473875,100.0,数据库原理
2,2022-03-01 17:02,测验,147392786,222820410,147473875,90.0,数据库原理
3,2022-03-01 17:05,测验,111948433,222820410,147473875,90.0,数据库原理
4,2022-03-01 17:03,测验,147383116,222820410,147473875,50.0,数据库原理


In [560]:
data_exams = pd.merge(data_ans, data_cs, on = '课程id')
print(data_exams.shape)

(2179, 8)


#### 数据保存

In [562]:
data_exams.to_csv('E:/比赛/data/2.课程测验成绩表_1.csv', index = False, encoding = 'utf_8_sig')
data_exam.to_csv('E:/比赛/data/2.课程测验成绩表.csv', index = False, encoding = 'utf_8_sig')

### 查看每个课程的每场测试的平均分以及平均花费时长（不考虑未测的分数）

In [43]:
# d_ex[d_ex['活动类型'] == '测验'].groupby(['课程id', '活动名'])[['分数', '参与时长']].mean()
data_exam[data_exam['活动类型'] == '测验'].groupby(['课程名', '活动id'])['分数'].mean()

课程名       活动id     
C语言程序设计B  142500        0.000000
          142501        0.000000
          142502        0.000000
中级财务会计    169391       72.135135
人工智能      889666       78.574542
                         ...    
高等数学B2    153837531    82.371542
          158710294    76.269841
          159733484    64.326531
          161961395    65.047233
          167770391    71.792829
Name: 分数, Length: 557, dtype: float64

### 查看每个课程的每个测验的参与情况（未测人数）

In [44]:
d_exs[d_exs['活动类型'] == '未测'].groupby(['课程id', '活动名'])['用户id'].count()

课程id       活动名         
222652756  向量代数与空间解析几何     3
           多元函数微分法及其应用     3
           微分方程            7
           无穷级数            9
           曲线积分与曲面积分       2
           重积分             2
222820410  单元测试2（2022上）    1
224894458  数学实验测试          3
Name: 用户id, dtype: int64

## 7. t_stat_job_finish --> 任务点完成记录表

In [45]:
# 超星数据
df_jfs = getdf(path, 't_stat_job_finish.xls')
data_jfs = getdata(df_jfs, df_jfs.columns[3:-2].values, ['日期', '用户id', '课程id', '活动id'])
data_jfs = dropdata_(data_jfs, '用户id', perids)
data_jfs.drop_duplicates(inplace = True)
print(data_jfs.shape)
data_jfs.head()

(2706, 9)
(2706, 4)
(2706, 4)


,日期,用户id,课程id,活动id
0,2022-02-24 20:39,147393474,222675513,1546839266729278
1,2022-02-24 20:54,147393474,222675513,1546839285359511
2,2022-02-24 21:02,147393474,222675513,1547524697809789
3,2022-02-24 21:04,147393474,222675513,1581777300573602
4,2022-02-24 21:15,147393474,222675513,1546837277339979


## 8. t_stat_course_job --> 课程任务表
### 147 行 10 列

In [46]:
df_cjs = getdf(path, 't_stat_course_job.xls')
data_cjs = getdata(df_cjs, ['job_id', 'name', 'type', 'courseid'], ['活动id', '任务点名称', '任务点类型', '课程id'])
data_cjs = data_str_clear(data_cjs)
data_cjs.drop_duplicates(inplace = True)
print(data_cjs.shape)
data_cjs.head()

(147, 10)
(147, 4)
(147, 4)


C:\Users\SUGA\AppData\Local\Temp\ipykernel_19160\1081961814.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = data[i].str.strip()
C:\Users\SUGA\AppData\Local\Temp\ipykernel_19160\3320121937.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cjs.drop_duplicates(inplace = True)


,活动id,任务点名称,任务点类型,课程id
0,1582119916176884,8.1堰流.mp4,VideoAttachment,222675513
1,1546839456877215,2.1静压强及其特性.mp4,VideoAttachment,222675513
2,154684034029472,3.1 液体运动的描述方法.mp4,VideoAttachment,222675513
3,1582119043467238,4.1 液流阻力和水头损失分类及计算公式.mp4,VideoAttachment,222675513
4,1546842378053993,量纲和谐原理.mp4,VideoAttachment,222675513


#### 数据合并

In [47]:
data_tasks = pd.merge(data_jfs, data_cjs, on = ['活动id', '课程id'])
data_tasks.reset_index(drop = True, inplace = True)
data_tasks.insert(1, '活动类型', '完成任务点')
data_tasks = df_clear(data_tasks)
print(data_tasks.shape)
data_tasks.head()

(2706, 7)


,日期,活动类型,用户id,课程id,活动id,任务点名称,任务点类型
0,2022-02-24 20:39,完成任务点,147393474,222675513,1546839266729278,1.2.1 液体的易流动性与惯性.mp4,VideoAttachment
1,2022-02-25 15:28,完成任务点,147389380,222675513,1546839266729278,1.2.1 液体的易流动性与惯性.mp4,VideoAttachment
2,2022-02-24 20:54,完成任务点,147393474,222675513,1546839285359511,1.2.2 液体的粘性.mp4,VideoAttachment
3,2022-02-25 15:36,完成任务点,147389380,222675513,1546839285359511,1.2.2 液体的粘性.mp4,VideoAttachment
4,2022-02-24 21:02,完成任务点,147393474,222675513,1547524697809789,1.2.3 压缩性和热胀性.mp4,VideoAttachment


#### 数据保存

In [563]:
data_tasks.to_csv('E:/比赛/data/5.课程任务点完成记录表_1.csv', index = False, encoding = 'utf_8_sig')

## 9. t_stat_log & t_stat_login --> 其他活动流水和登录流水

In [49]:
# 获得所有文件路径
log_file = FilesPath('data/t_stat_log&t_stat_login/')

data_log = pd.DataFrame()
for file_path in (log_file):
    dlog = pd.read_csv(file_path, encoding = 'UTF-8')
    data_log = pd.concat([data_log, dlog])

# 重新设置索引 从0开始
data_log.reset_index(drop = True, inplace = True)

In [50]:
data_str_clear(data_log)
data_logs = data_log[[*data_log.columns[3:6], 'type']]
data_logs.columns = ['日期', '用户id', '课程id', '活动类型']
data_logs = data_logs.drop(data_logs[data_logs['用户id'] == 0].index)
data_logs.reset_index(drop = True, inplace = True)

In [51]:
indexs = []
for indx,id1 in zip(data_logs.index, data_logs['用户id'].values):
    if id1 not in data_stu['用户id'].to_list():
        indexs.append(indx)
    else:
        continue
datalogs = data_logs.drop(indexs)

自主学习：
accesslog 登录访问
course pv 访问课程

完成任务点：
jobfinish 完成任务点

完成作业：
setwork 提交作业

讨论：
bbslog 讨论
bbslog-replydel 回复讨论


互动：
piyuework 批阅互评作业

workanswer 查阅作业答案
workscore 查阅作业分数
piyueworkevaluation 查阅作业评价

examanswer 查阅试卷答案
examscore 查阅考试成绩

ceyaanswer 查阅测验答案
ceyascore 查阅测验分数

In [52]:
k1 = []
ins1 = []
ins2 = []
for i in datalogs['活动类型'].unique():
    try:
        if (i not in [*data['活动类型'].unique(), 'AttendLog']) and (i[0:9] == 'AttendLog'):
            ins1 += datalogs[datalogs['活动类型'] == i].index.to_list()
            datalogs.loc[datalogs['活动类型'] == i, '活动类型'] = int(i[9:])
        else:
            if (i not in data['活动类型'].unique()) and i[0:9] != 'AttendLog':
                ins2 += datalogs[datalogs['活动类型'] == i].index.to_list()
                k1.append(i)
    except:
        continue

In [53]:
v1 = ['登录访问', '查阅作业答案', '查阅作业分数', '完成任务点', '查阅试卷答案', ' 查阅考试成绩', '查阅测验答案', '查阅测验分数', '讨论', '查阅作业评价', '访问课程', '批阅互评作业', '提交作业', '回复讨论']
dtps = dict(zip(k1, v1))
# print(dtp)
datalogs.loc[ins1, '活动类型'] = datalogs.loc[ins1, '活动类型'].map(sets)
datalogs.loc[ins2, '活动类型'] = datalogs.loc[ins2, '活动类型'].map(dtps)
datalogs['活动id'] = '其他'
datalogs.drop_duplicates(inplace = True)
print(datalogs.shape)
datalogs.reset_index(drop = True, inplace = True)
print(datalogs.shape)
datalogs.head()

(73009, 5)
(73009, 5)


,日期,用户id,课程id,活动类型,活动id
0,2022-02-05 11:13:27,147392719,222595437,登录访问,其他
1,2022-02-05 11:13:59,147392719,222595437,登录访问,其他
2,2022-02-05 12:00:10,111946208,222595437,登录访问,其他
3,2022-02-08 11:07:26,147392719,222595437,登录访问,其他
4,2022-02-09 10:53:21,111949220,222595437,登录访问,其他


#### 雨课堂数据

In [54]:
# 雨课堂数据
df_jfb = getdf(path, 'b1_t_stat_log.xlsx')
data_jfb = querydate(df_jfb, dates[0], dates[1], ['last_modify_time', 'personid', 'clazzid', 'type', 'logid'], ['日期', '用户id', '班级id', '活动类型', '活动id'])
data_jfb = data_jfb[data_jfb['用户id'].isna() == False]
data_jfb = dropdata_(data_jfb, '用户id', peridb)
data_jfb = dropdata_(data_jfb, '用户id', 0)
data_jfb.head()

(169673, 11)


,日期,用户id,班级id,活动类型,活动id
0,2022-08-21 00:33:30,31719382.0,8009860,piyuework,182831528415585
1,2022-08-21 00:33:30,31719382.0,8009864,piyuework,182833128415602
2,2022-08-21 00:33:30,38182318.0,8008779,piyueexam,18675428420962
3,2022-08-21 00:33:30,38182318.0,8008779,piyueexam,19054828421028
4,2022-08-21 00:33:30,38182318.0,8008779,piyueexam,20910528421137


In [55]:
data_njfb = data_jfb[data_jfb.活动类型 == 'examanswer']
data_njfb.loc[:, '活动类型'] = '查阅试卷答案'
data_njfb.reset_index(drop = True, inplace = True)
data_njfb.drop_duplicates(inplace = True)
print(data_njfb.shape)
data_njfb.head()

(35373, 5)


C:\Users\SUGA\AppData\Local\Temp\ipykernel_19160\3006673725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_njfb.loc[:, '活动类型'] = '查阅试卷答案'
C:\Users\SUGA\AppData\Local\Temp\ipykernel_19160\3006673725.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_njfb.drop_duplicates(inplace = True)


,日期,用户id,班级id,活动类型,活动id
0,2022-08-26 05:01:16,28433403.0,8010936,查阅试卷答案,28433403176738
1,2022-08-26 05:01:23,28432273.0,8011490,查阅试卷答案,28432273172158
2,2022-08-26 05:00:58,28432273.0,8011492,查阅试卷答案,28432273172159
3,2022-08-26 05:01:29,28432273.0,8011492,查阅试卷答案,28432273179979
4,2022-08-26 05:01:11,28432273.0,8011490,查阅试卷答案,28432273179978


In [56]:
data_jfbb = pd.merge(data_njfb, data_cla, on = '班级id')
data_jfbb = data_jfbb[['日期', '用户id', '课程id', '活动类型', '活动id']]
data_jfbb.reset_index(drop = True, inplace = True)
data_jfbb['用户id'] = data_jfbb['用户id'].astype(int)
print(data_jfbb.shape)
data_jfbb.head()

(32946, 5)


,日期,用户id,课程id,活动类型,活动id
0,2022-08-26 05:01:16,28433403,2260721,查阅试卷答案,28433403176738
1,2022-08-26 05:01:16,28433471,2260721,查阅试卷答案,28433471182902
2,2022-08-26 05:01:14,28413048,2260721,查阅试卷答案,28413048189188
3,2022-08-26 05:01:19,28433318,2260721,查阅试卷答案,28433318189188
4,2022-08-26 05:01:11,28425227,2260721,查阅试卷答案,28425227189188


#### 数据合并

In [57]:
newdatalog = pd.concat([datalogs, data_jfbb], axis = 0)
newdatalog = newdatalog[['日期', '活动类型', '用户id', '课程id', '活动id']]
newdatalog = newdatalog[newdatalog['活动类型'].isna() == False]
newdatalog.reset_index(drop = True, inplace = True)
print(newdatalog.shape)
newdatalog.head()

(105950, 5)


,日期,活动类型,用户id,课程id,活动id
0,2022-02-05 11:13:27,登录访问,147392719,222595437,其他
1,2022-02-05 11:13:59,登录访问,147392719,222595437,其他
2,2022-02-05 12:00:10,登录访问,111946208,222595437,其他
3,2022-02-08 11:07:26,登录访问,147392719,222595437,其他
4,2022-02-09 10:53:21,登录访问,111949220,222595437,其他


#### 保存数据

In [564]:
newdatalog.to_csv('E:/比赛/data/6.其他活动流水和登录流水.csv', index = False, encoding = 'utf_8_sig')

## 10. t_stat_student_score --> 学生成绩表

In [59]:
# 超星数据
df_scs = pd.read_excel('data/t_stat_student_score.xls')
print(df_scs.shape)
# df_scs.head(5)

(2248, 9)


In [60]:
data_scs = getdata(df_scs, ['last_modify_time', 'personid', 'courseid', 'score'], ['日期', '用户id', '课程id', '综合成绩'])
data_scs = dropdata_(data_scs, '用户id', perids)

(2248, 4)


In [61]:
# 结合实际情况，将综合成绩小于25都作为缺省值处理
import numpy as np
insmea = []
for i in data_scs['课程id'].unique():
    mea = data_scs[(data_scs['课程id'] == i) & (data_scs['综合成绩'] > 5)]['综合成绩'].mean()
    if mea is np.nan:
        data_scs = data_scs.drop(data_scs[data_scs['课程id'] == i].index)
    elif mea < 30:
        data_scs.loc[data_scs['课程id'] == i, '综合成绩'] = (100 - data_scs.loc[data_scs['课程id'] == i, '综合成绩']).round(2)
    elif mea < 40:
        data_scs.loc[data_scs['课程id'] == i, '综合成绩'] = (90 - data_scs.loc[data_scs['课程id'] == i, '综合成绩']).round(2)
    elif mea < 60:
        data_scs.loc[(data_scs['课程id'] == i) & (data_scs['综合成绩'] < 55), '综合成绩'] = (45 + data_scs.loc[data_scs['课程id'] == i, '综合成绩']).round(2)
    else:
        data_scs.loc[(data_scs['课程id'] == i) & (data_scs['综合成绩'] <= 25), '综合成绩'] = mea.round(2)

In [62]:
print('处理后平均值: ',data_scs.综合成绩.mean())

data_scs.insert(1,'活动类型', '综合考试')
data_scs.insert(4,'活动id', '考试')
print(data_scs.shape)
data_scs.head()

处理后平均值:  85.20765710799266
(1639, 6)


,日期,活动类型,用户id,课程id,活动id,综合成绩
93,2022-02-22 17:00,综合考试,51465331,223018597,考试,88.09
94,2022-03-03 07:41,综合考试,67473135,223018597,考试,100.00
95,2022-02-22 17:00,综合考试,69984054,223018597,考试,92.42
96,2022-02-22 17:00,综合考试,111943347,223018597,考试,88.68
97,2022-02-22 17:00,综合考试,111944035,223018597,考试,94.25


#### 各课程的综合成绩平均值

In [63]:
for i in data_scs.groupby('课程id')['综合成绩']:
    print(i[0], i[1].mean())

222602451 73.40425531914893
222652756 87.72106824925817
222675513 95.21153846153847
222807286 77.55718562874252
222808035 85.45975000000001
222820410 58.48226804123711
222820426 97.9555
223018597 87.75105590062113
223945300 90.98916666666666
224841013 91.18518518518519
224894458 96.36381868131868
224922228 85.44764705882349


#### 雨课堂数据

In [64]:
# 雨课堂数据
df_scb = pd.read_excel('data/b1_t_stat_student_score.xlsx')
data_scb = querydate(df_scb, dates[0], dates[1], ['last_modify_time', 'personid', 'clazzid', 'score'], ['日期', '用户id', '班级id', '综合成绩'])
data_scb = dropdata_(data_scb, '用户id', perids)

#### 填充缺省值

In [65]:
# 将综合成绩均值小于5的课程的考试记录删除默认没有该项操作，对均值小于20、30、40、60的各类区间的每门课程的考试记录的综合成绩进行缺省值补充
for i in data_scb.groupby(['班级id'])['综合成绩']:
    if i[1].mean() < 5:
        data_scb = data_scb.drop(i[1].index.to_list())
    elif i[1].mean() < 20:
        data_scb.loc[data_scb['班级id'] == i[0], '综合成绩'] = (90 - data_scb.loc[data_scb['班级id'] == i[0], '综合成绩'])
    elif i[1].mean() < 30:
        data_scb.loc[data_scb['班级id'] == i[0], '综合成绩'] = (95 - data_scb.loc[data_scb['班级id'] == i[0], '综合成绩'])
    elif i[1].mean() < 40:
        data_scb.loc[data_scb['班级id'] == i[0], '综合成绩'] = (100 - data_scb.loc[data_scb['班级id'] == i[0], '综合成绩'])
    elif i[1].mean() < 60:
        data_scb.loc[(data_scb['班级id'] == i[0]) & (data_scb['综合成绩'] < 60), '综合成绩'] = (30 + data_scb.loc[data_scb['班级id'] == i[0], '综合成绩'])

In [66]:
data_scb.insert(1,'活动类型', '综合考试')
data_scb.insert(4,'活动id', '考试')
print(data_scb.shape)
data_scb.head()

(28076, 6)


,日期,活动类型,用户id,班级id,活动id,综合成绩
61,2022-06-10 05:01:25,综合考试,28405855,8009388,考试,63.83
95,2022-06-10 05:02:35,综合考试,28405870,8009449,考试,87.86
96,2022-04-25 05:19:48,综合考试,28405870,8009479,考试,87.65
123,2022-03-23 05:00:23,综合考试,28405892,8010139,考试,82.62
154,2022-03-23 05:00:23,综合考试,28405925,8010141,考试,78.62


In [67]:
data_nscb = pd.merge(data_scb, data_cla, on = '班级id')
data_nscb = data_nscb[['日期', '活动类型', '用户id', '课程id', '活动id', '综合成绩']]
data_nscb.reset_index(drop = True, inplace = True)
print(data_nscb.shape)
data_nscb.head()

(27199, 6)


,日期,活动类型,用户id,课程id,活动id,综合成绩
0,2022-06-10 05:01:25,综合考试,28405855,1944089,考试,63.83
1,2022-06-10 05:01:25,综合考试,28406687,1944089,考试,63.83
2,2022-06-10 05:01:25,综合考试,28412716,1944089,考试,61.75
3,2022-06-10 05:02:33,综合考试,28420546,1944089,考试,75.17
4,2022-06-10 05:02:35,综合考试,28425138,1944089,考试,83.83


#### 数据合并

In [68]:
data_sc = pd.concat([data_scs, data_nscb], axis = 0)
data_sc.reset_index(drop = True, inplace = True)
print(data_sc.shape)
data_sc.head()

(28838, 6)


,日期,活动类型,用户id,课程id,活动id,综合成绩
0,2022-02-22 17:00,综合考试,51465331,223018597,考试,88.09
1,2022-03-03 07:41,综合考试,67473135,223018597,考试,100.00
2,2022-02-22 17:00,综合考试,69984054,223018597,考试,92.42
3,2022-02-22 17:00,综合考试,111943347,223018597,考试,88.68
4,2022-02-22 17:00,综合考试,111944035,223018597,考试,94.25


#### 数据保存

In [565]:
data_sc.to_csv('E:/比赛/data/8.学生综合考试成绩表.csv', index = False, encoding = 'utf_8_sig')

## 11.  t_stat_widget_log --> 课堂活动发布表
### 每门课程每项活动的完成情况

In [70]:
# 每门课程每项活动的完成情况
# df_w = getdf(path, 't_stat_widget_log.xls')
# data_w = getdata(df_w, ['courseid', 'activity_id', 'dtype', 'send_to_student', 'attend_count'], ['课程id', '活动id', '活动类型', '应参人数', '实参人数'])
# data_w['活动类型'] = data_w['活动类型'].map(acts)
# data_w.head()

## 12. t_stat_work_answer --> 学生作业提交记录表

In [71]:
df_wa = getdf(path, 't_stat_work_answer.xls')
data_wa = getdata(df_wa, ['answer_time', 'personid', 'courseid', 'work_id', 'status', 'score'], ['日期', '用户id', '课程id', '活动id', '状态', '成绩'])
data_wa = dropdata_(data_wa, '用户id', perids)
data_wa = dropdata_(data_wa, '状态', [0, 1])
data_wa.pop('状态')
data_wa.insert(1, '活动类型', '完成作业')
data_wa.head()

(5519, 17)
(5519, 6)


,日期,活动类型,用户id,课程id,活动id,成绩
0,2022-03-22 23:06,完成作业,51465331,223018597,52245548afa948c0b83e8c644ff39dd3,95.0
1,2022-03-21 10:33,完成作业,111943347,223018597,52245548afa948c0b83e8c644ff39dd3,100.0
2,2022-03-20 19:31,完成作业,111944914,223018597,52245548afa948c0b83e8c644ff39dd3,65.0
3,2022-03-21 09:26,完成作业,111945078,223018597,52245548afa948c0b83e8c644ff39dd3,85.0
4,2022-03-20 15:13,完成作业,111945359,223018597,52245548afa948c0b83e8c644ff39dd3,95.0


#### 保存数据

In [566]:
data_wa.to_csv('E:/比赛/data/7.学生课程作业成绩表_1.csv', index = False, encoding = 'utf_8_sig')